### RankForecasting-shortterm-paper
basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlapstatus = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [3]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018%s-nocarid-context40'%('-noinlap' if inlapmode==0 else '-inlapstatus')
    
    if runts == 'RANK':
        acc, ret = simulation(dataset, testevent, 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                      pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
    else:
        acc, ret = simulation(dataset, testevent, 
                        'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)

    #print(np.mean(acc, axis=0))
    return acc, ret

In [4]:
def plot_prob_forecasts_ex(ts_entry, forecast_entry, ts2, outputfile):

    plot_length = int(forecast_entry[0].samples.shape[1] *1.2) 
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    figcnt = len(ts_entry)
    for idx in range(figcnt):
        fig, axs = plt.subplots(1, 1, figsize=(20, 7))
        #ts_entry[idx][-plot_length:].plot(ax=axs)  # plot the time series
        #forecast_entry[idx].plot(prediction_intervals=prediction_intervals, color='g')
        ts_entry[idx].iloc[-plot_length:,0].plot(ax=axs, linewidth=1)  # plot the time series
        
        ts2.iloc[-plot_length:,0].plot(ax=axs, linewidth=1, label='CurRank')  # plot the time series
        
        forecast_entry[idx].copy_dim(0).plot(prediction_intervals=prediction_intervals, color='g')
        
        plt.ylabel('Rank position')
        plt.xlabel('Lap')
        
        laps = range(1, 200, 20)
        
        #axs.set_xticks(laps)
        #axs.set_xticklabels([1+x*30 for x in range(7)])
        #axs.set_xticklabels(laps)
        
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.savefig(outputfile + '-%d.pdf'%idx)
        plt.show()

In [5]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

### test low mode

In [6]:
acc, ret = {}, {}

In [7]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1,inlapmode=0,featuremode = stint.FEATURE_PITAGE)
_ = prisk(ret[mid][0][1], ret[mid][0][2])
mid = 'oracle-2019'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1,inlapmode=0,featuremode = stint.FEATURE_PITAGE)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
start pitlap: 10
update lapstatus done.
--------- simu_onestep: 10-12, endpos=13: 10 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044]
[15.11400032 15.54940033]
simulation done: 33
start pitlap: 11
update lapstatus done.
--------- simu_onestep: 11-13, endpos=14: 11 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032]
[15.54940033 16.14319992]
simulation done: 33
start pitlap: 12
update lapstatus done.
--------- simu_onestep: 12-14, endpos=15: 12 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033]
[16.14319992 16.76650047]
simulation done: 33
start pitlap: 13
update l

simulation done: 33
start pitlap: 31
update lapstatus done.
--------- simu_onestep: 31-33, endpos=34: 31 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964]
[42.99969864 44.07400131]
simulation done: 33
start pitlap: 32
update lapstatus done.
--------- simu_onestep: 32-34, endpos=35: 32 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.

simulation done: 31
start pitlap: 45
update lapstatus done.
--------- simu_onestep: 45-47, endpos=48: 45 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964 42.99969864 44.07400131 27.26370049 27.77210045
 28.26370049 28.47809982 28.52879906 28.3696003  28.07749939 27.45960045
 27.51569939 27.40550041 27.46339989 27.92230034]
[33.11600113 43.91939926]
simulation done: 31
start pitlap: 46
update lapstatus done.
--------- simu_onestep: 46-48, endpos=49: 46 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 1

simulation done: 30
start pitlap: 56
update lapstatus done.
--------- simu_onestep: 56-58, endpos=59: 56 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964 42.99969864 44.07400131 27.26370049 27.77210045
 28.26370049 28.47809982 28.52879906 28.3696003  28.07749939 27.45960045
 27.51569939 27.40550041 27.46339989 27.92230034 33.11600113 43.91939926
 20.27639961 21.80979919 11.80739975 11.06540012 13.06309986 16.14669991
  4.14669991  4.78840017  7.09969997]
[13.17790031 10.86260033]
simulation done: 30
start pitlap: 57
update lapstatus done.
--------- simu_onestep: 57-59, endpos=60: 57 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  

...

simulation done: 30
start pitlap: 145
update lapstatus done.
simulation done: 30
start pitlap: 146
update lapstatus done.
simulation done: 30
start pitlap: 147
update lapstatus done.
simulation done: 30
start pitlap: 148
update lapstatus done.
simulation done: 30
start pitlap: 149
update lapstatus done.
simulation done: 30
start pitlap: 150
update lapstatus done.
simulation done: 30
start pitlap: 151
update lapstatus done.
simulation done: 30
start pitlap: 152
update lapstatus done.
simulation done: 30
start pitlap: 153
update lapstatus done.
simulation done: 30
start pitlap: 154
update lapstatus done.
simulation done: 30
start pitlap: 155
update lapstatus done.
simulation done: 30
start pitlap: 156
update lapstatus done.
simulation done: 30
start pitlap: 157
update lapstatus done.
simulation done: 30
start pitlap: 158
update lapstatus done.
simulation done: 30
start pitlap: 159
update lapstatus done.
simulation done: 30
start pitlap: 160
update lapstatus done.
simulation done: 30
star

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.74}, mae={1.08}, rmse={2.44},r2={0.92}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.7393617  1.0794102  2.43625427 0.92248556]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 51.60it/s]

0.047332869165842166 0.07452331220149755 0.036432262083623435


In [12]:
mid = 'oracle-2018-1'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1,inlapmode=1,featuremode = stint.FEATURE_PITAGE)
_ = prisk(ret[mid][0][1], ret[mid][0][2])
mid = 'oracle-2019-1'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1,inlapmode=1,featuremode = stint.FEATURE_PITAGE)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
start pitlap: 10
update lapstatus done.
--------- simu_onestep: 10-12, endpos=13: 10 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044]
[15.11400032 15.54940033]
simulation done: 33
start pitlap: 11
update lapstatus done.
--------- simu_onestep: 11-13, endpos=14: 11 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032]
[15.54940033 16.14319992]
simulation done: 33
start pitlap: 12
update lapstatus done.
--------- simu_onestep: 12-14, endpos=15: 12 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033]
[16.14319992 16.76650047]
simulation done: 33
start pitlap: 13
update l

simulation done: 33
start pitlap: 31
update lapstatus done.
--------- simu_onestep: 31-33, endpos=34: 31 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964]
[42.99969864 44.07400131]
simulation done: 33
start pitlap: 32
update lapstatus done.
--------- simu_onestep: 32-34, endpos=35: 32 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.

simulation done: 31
start pitlap: 45
update lapstatus done.
--------- simu_onestep: 45-47, endpos=48: 45 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964 42.99969864 44.07400131 27.26370049 27.77210045
 28.26370049 28.47809982 28.52879906 28.3696003  28.07749939 27.45960045
 27.51569939 27.40550041 27.46339989 27.92230034]
[33.11600113 43.91939926]
simulation done: 31
start pitlap: 46
update lapstatus done.
--------- simu_onestep: 46-48, endpos=49: 46 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 1

simulation done: 30
start pitlap: 56
update lapstatus done.
--------- simu_onestep: 56-58, endpos=59: 56 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  9.22929955  9.95069981
 11.31040001 13.09840012 13.69219971 14.2531004  14.68080044 15.11400032
 15.54940033 16.14319992 16.76650047 17.44070053 18.33169937 19.1692009
 20.31669998 21.52330017 22.5291996  23.27549934 24.29249954 26.6697998
 27.68729973 27.68930054 28.09490013 28.16290092 28.47019958 26.1012001
 23.33919907 20.39579964 42.99969864 44.07400131 27.26370049 27.77210045
 28.26370049 28.47809982 28.52879906 28.3696003  28.07749939 27.45960045
 27.51569939 27.40550041 27.46339989 27.92230034 33.11600113 43.91939926
 20.27639961 21.80979919 11.80739975 11.06540012 13.06309986 16.14669991
  4.14669991  4.78840017  7.09969997]
[13.17790031 10.86260033]
simulation done: 30
start pitlap: 57
update lapstatus done.
--------- simu_onestep: 57-59, endpos=60: 57 ----------
[ 5.30070019  6.69729996  7.73920012  8.52789974  

...

simulation done: 30
start pitlap: 144
update lapstatus done.
simulation done: 30
start pitlap: 145
update lapstatus done.
simulation done: 30
start pitlap: 146
update lapstatus done.
simulation done: 30
start pitlap: 147
update lapstatus done.
simulation done: 30
start pitlap: 148
update lapstatus done.
simulation done: 30
start pitlap: 149
update lapstatus done.
simulation done: 30
start pitlap: 150
update lapstatus done.
simulation done: 30
start pitlap: 151
update lapstatus done.
simulation done: 30
start pitlap: 152
update lapstatus done.
simulation done: 30
start pitlap: 153
update lapstatus done.
simulation done: 30
start pitlap: 154
update lapstatus done.
simulation done: 30
start pitlap: 155
update lapstatus done.
simulation done: 30
start pitlap: 156
update lapstatus done.
simulation done: 30
start pitlap: 157
update lapstatus done.
simulation done: 30
start pitlap: 158
update lapstatus done.
simulation done: 30
start pitlap: 159
update lapstatus done.
simulation done: 30
star

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

simulation done: 17
model: acc={0.82}, mae={0.95}, rmse={2.24},r2={0.93}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.82446809 0.95079055 2.24086944 0.93442014]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 61.69it/s]

0.04015537396020375 0.06858685428805257 0.035389106721878165


In [13]:
acc

{'oracle-2018': array([[[0.79787234, 1.10749064, 2.42768057, 0.91771522],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-2019': array([[[0.7393617 , 1.0794102 , 2.43625427, 0.92248556],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-2018-1': array([[[0.83510638, 0.95018727, 2.16967524, 0.93427571],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-2019-1': array([[[0.82446809, 0.95079055, 2.24086944, 0.93442014],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]])}

In [18]:
mid = 'oracle-2018-timediff-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'TIMEDIFF', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE)
mid = 'oracle-2019-timediff-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'TIMEDIFF', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.77}, mae={1.41}, rmse={3.16},r2={0.86}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.77}, mae={1.47}, rmse={3.28},r2={0.85}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.77}, mae={1.45}, rmse={3.24},r2={0.85}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.76950355 1.44357054 3.22475437 0.85477977]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.71}, mae={1.37}, rmse={3.21},r2={0.87}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.69}, mae={1.39}, rmse={3.18},r2={0.87}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.71}, mae={1.38}, rmse={3.14},r2={0.87}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.70567376 1.38023332 3.17738901 0.86813806]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [19]:
mid = 'oracle-2018-rank-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE)
mid = 'oracle-2019-rank-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.76}, mae={1.24}, rmse={3.01},r2={0.87}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.74}, mae={1.27}, rmse={3.06},r2={0.87}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.76}, mae={1.23}, rmse={2.98},r2={0.88}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.75177305 1.24581773 3.01363257 0.87318439]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.80}, mae={1.11}, rmse={2.88},r2={0.89}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.77}, mae={1.12}, rmse={2.92},r2={0.89}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.78}, mae={1.10}, rmse={2.90},r2={0.89}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.78191489 1.11103215 2.90010919 0.89015423]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [17]:
acc

{'oracle-2018': array([[[0.79787234, 1.10749064, 2.42768057, 0.91771522],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-2019': array([[[0.7393617 , 1.0794102 , 2.43625427, 0.92248556],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-2018-1': array([[[0.83510638, 0.95018727, 2.16967524, 0.93427571],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-2019-1': array([[[0.82446809, 0.95079055, 2.24086944, 0.93442014],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-2018-timediff-pitmodel-inlap': array([[[0.75      , 1.47340824, 3.30395292, 0.84759327],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]],
 
        [[0.77659574, 1.43220974, 3.19532664, 0.85745009],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]],
 
        [[0.76595745, 1.43595506, 3.22356724, 0.85491921],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-2019-timediff-pitmodel-inlap': array([[[0

In [9]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])
mid = 'oracle-2019'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'deepar-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1, model='deepAR')
_ = prisk(ret[mid][0][1], ret[mid][0][2])
mid = 'deepar-2019'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1, model='deepAR')
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK')
test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK')

In [ ]:
mid = 'oracle-2018-timediff-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'TIMEDIFF', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE))
mid = 'oracle-2019-timediff-pitmodel-inlap'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'TIMEDIFF', loopcnt=3,inlapmode=1,featuremode = stint.FEATURE_PITAGE))

#### =====================================================

In [ ]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

In [ ]:
mid = 'oracle-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'deepar-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'currank-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1,model='naive')
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2018'
full_samples,full_tss = ret[mid][0][1], ret[mid][0][2]
testcar = 12
freq = '1min'
start = pd.Timestamp("01-01-2019", freq=freq) 
fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)
index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
ts = pd.DataFrame(index = index, data = full_tss[testcar])

tss2 = np.zeros_like(full_tss[testcar])
tss2[2:] = full_tss[testcar][:-2]
index2 = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))

ts2 = pd.DataFrame(index = index2, data = tss2)

plot_prob_forecasts_ex([ts], [fc], ts2,'output-car12')

In [ ]:
%debug

In [ ]:
ret.keys()

### save result df

In [ ]:
df = ret['oracle-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_oracle.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')